# Load Libaries

In [ ]:
import pandas as pd
import numpy as np
from sklearn.svm import LinearSVC
from nltk.tokenize import TreebankWordTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import cross_validate
from sklearn import metrics
from sklearn.feature_selection import VarianceThreshold
from sklearn import preprocessing
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
import sys
import argparse
import csv
from scipy.sparse import *
import pandas as pd
import regex 
from sklearn.base import TransformerMixin
from nltk.tokenize import TreebankWordTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from nltk.util import ngrams
from sklearn.model_selection import StratifiedKFold, train_test_split


In [ ]:
from keras.models import Model, Sequential
from keras.layers import Dense, Input, Dropout, LSTM, Activation,GRU
from keras.layers import Bidirectional, GlobalMaxPool1D,Concatenate,Conv1D,SpatialDropout1D,BatchNormalization,concatenate
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.initializers import glorot_uniform
from keras.layers import Activation, Permute, RepeatVector, Lambda
from keras.layers import Dense, Dropout, Flatten, LSTM, GRU, Bidirectional, Input, Multiply
from keras.layers import Conv1D, MaxPooling1D, GlobalMaxPooling1D
from tensorflow.keras import regularizers


from keras.optimizers import Adam

from matplotlib import pyplot as plt
import tensorflow as tf
%matplotlib inline

In [ ]:
import re

In [ ]:
!pip install emoji==0.6.0

In [ ]:
import emoji


In [ ]:
from collections import Counter, OrderedDict


In [ ]:
import math

In [ ]:
import random
from random import seed

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder


In [ ]:
from keras import backend as K


In [ ]:
from  sklearn.metrics import f1_score,accuracy_score,precision_score,classification_report
from sklearn.metrics import confusion_matrix


# Helper Functions

In [ ]:
def clean_text(text):  
    # ref: https://github.com/bakrianoo/aravec
    search = ["أ","إ","آ","ة","_","-","/",".","،"," و "," يا ",'"',"ـ","'","ى",
              "\\",'\n', '\t','&quot;','?','؟','!']
    replace = ["ا","ا","ا","ه"," "," ","","",""," و"," يا",
               "","","","ي","",' ', ' ',' ',' ? ',' ؟ ', ' ! ']
    
    tashkeel = re.compile(r'[\u0617-\u061A\u064B-\u0652]')
    text = re.sub(tashkeel,"", text)
    
    # longation = re.compile(r'(.)\1+')
    # subst = r"\1\1"
    # text = re.sub(longation, subst, text)
    
    text = re.sub(r"[^\w\s]", '', text)
    text = re.sub(r"[a-zA-Z]", '', text)
    text = re.sub(r"\d+", ' ', text)
    text = re.sub(r"\n+", ' ', text)
    text = re.sub(r"\t+", ' ', text)
    text = re.sub(r"\r+", ' ', text)
    text = re.sub(r"\s+", ' ', text)
    text = text.replace('وو', 'و')
    text = text.replace('يي', 'ي')
    text = text.replace('اا', 'ا')
    
    for i in range(0, len(search)):
        text = text.replace(search[i], replace[i])
    
    text = text.strip()
    
    return text

In [ ]:
from gensim.models import KeyedVectors
def load_w2v(filepath,binary):
    return KeyedVectors.load_word2vec_format(filepath, binary=binary)

In [ ]:
# Get the vec representation of a set of tweets based on a specified embedding (can be a word or emoji mapping)
def get_tweets_embeddings(tweets, vec_map, embedding_dim=100, init_unk=False, variance=None, weighted_average=True):
    # Get the variance of the embedding map
    if init_unk and variance is None:
        variance = embedding_variance(vec_map)
        print("Vector mappings have variance ", variance)
    # If set, calculate the tf-idf weight of each embedding, otherwise, no weighting (all weights are 1.0)
    if weighted_average:
        weights = get_tf_idf_weights(tweets, vec_map)
    else:
        weights = {k: 1.0 for k in vec_map.vocab}
    tw_emb = np.zeros((len(tweets), embedding_dim))
    for i, tw in enumerate(tweets):
        total_valid = 0
        for word in tw.split():
            embedding_vector=None 
            if word in vec_map.wv:
              embedding_vector = vec_map[word]
            if embedding_vector is not None:
                tw_emb[i] = tw_emb[i] + embedding_vector * weights[word]
                total_valid += 1
            elif init_unk:
                seed(1337603)
                tw_emb[i] = np.random.uniform(-variance, variance, size=(1, embedding_dim))
            # else:
            #    print("Not found: ", word)
        # Get the average embedding representation for this tweet
        tw_emb[i] /= float(max(total_valid, 1))
    return tw_emb


In [ ]:
# Calculate the variance of an embedding (like glove, word2vec, emoji2vec, etc)
# Used to sample new uniform distributions of vectors in the interval [-variance, variance]
def embedding_variance(vec_map):
    variance = np.sum([np.var(vec) for vec in vec_map.vectors]) / len(vec_map.vectors)
    return variance


In [ ]:
# Get some idea about the max and mean length of the tweets (useful for deciding on the sequence length)
def get_max_len_info(tweets, average=False):
    sum_of_length = sum([len(l.split()) for l in tweets])
    avg_tweet_len = sum_of_length / float(len(tweets))
    print("Mean of train tweets: ", avg_tweet_len)
    max_tweet_len = len(max(tweets, key=len).split())
    print("Max tweet length is = ", max_tweet_len)
    if average:
        return avg_tweet_len
    return max_tweet_len

In [ ]:
# Extract each tweet's emojis - obv. it's just a brute force solution (so, it's slow) but works in ALL cases
def extract_emojis(tweets):
    emojis = []
    for tw in tweets:
        tw_emojis = []
        for word in tw:
            chars = list(word)
            for ch in chars:
                # print(ch)
                if ch in emoji.UNICODE_EMOJI:
                    tw_emojis.append(ch)
        emojis.append(' '.join(tw_emojis))
    return emojis

def contain_emoji(tweets):

  emoji___=''.join(c for c in tweets if c in emoji.UNICODE_EMOJI)  
  if len(emoji___)>0:
    return True
  return False

In [ ]:
def encode_text_as_word_indexes(train_tweets, test_tweets , max_num_words=None, lower=False, char_level=False):
    # Create the tokenizer
    tokenizer = Tokenizer(num_words=max_num_words, filters='', lower=lower, split=" ", char_level=char_level)
    # Fit the tokenizer on the documents
    tokenizer.fit_on_texts(train_tweets)
    # Encode each example as a sequence of word indexes based on the vocabulary of the tokenizer
    x_train = tokenizer.texts_to_sequences(train_tweets)
    x_test=tokenizer.texts_to_sequences(test_tweets)
   
    return tokenizer, x_train,x_test

In [ ]:
def convert_emojis(text):
    emoji___=''.join(c for c in text if c in emoji.UNICODE_EMOJI)
    return emoji.demojize(emoji___)

In [ ]:
def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

In [ ]:
def transform(df):
    
    df["word_count"] = df["cleaned_text"].apply(lambda x: len(x.split()))
    
    df["contain_emoji"] = False
    
    df["demoji_text"] = ""
    for index, row in df.iterrows():
      df.loc[index, 'cleaned_text']=remove_urls(row["cleaned_text"])
      if contain_emoji(row["tweet"]):
        df.loc[index, 'contain_emoji']=True
        df.loc[index, 'demoji_text']=convert_emojis(row["tweet"])    
    return df



In [ ]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [ ]:
# Compute the word-embedding matrix
def get_embedding_matrix(word2vec_map, word_to_index, embedding_dim, init_unk=True, variance=None):
    # Get the variance of the embedding map
    if init_unk and variance is None:
        variance = embedding_variance(word2vec_map)
        print("Word vectors have variance ", variance)
    # Initialize the embedding matrix as a numpy array of zeros of shape (vocab_len, dimensions of word vectors)
    embedding_matrix = np.zeros((len(word_to_index) + 1, embedding_dim))
    for word, i in word_to_index.items():
        embedding_vector=None 
        if word in word2vec_map.wv:
          embedding_vector = word2vec_map[word]

        # embedding_vector = word2vec_map.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
        elif init_unk:
            # Unknown tokens are initialized randomly by sampling from a uniform distribution [-var, var]
            seed(1337603)
            embedding_matrix[i] = np.random.uniform(-variance, variance, size=(1, embedding_dim))
        # else:
        #    print("Not found: ", word)
    return embedding_matrix

In [ ]:
def keras_dl( word2vec_embegging,emoji_embedd,voacb_shape_embdd,input_shape_1,emoji_shape ):   
    inp = Input(shape = (input_shape_1,), name = 'text_clean')
    x = Embedding(voacb_shape_embdd,300,weights = [word2vec_embegging], trainable = False)(inp)
    
    x_lstm = Bidirectional(LSTM(128, return_sequences = True))(x)
    x_lstm_c1d = Conv1D(64,kernel_size=3,padding='valid',activation='tanh')(x_lstm)
    x_lstm_c1d_gp = GlobalMaxPooling1D()(x_lstm_c1d)
    
    x_gru = Bidirectional(GRU(128, return_sequences = True))(x)
    x_gru_c1d = Conv1D(64,kernel_size=2,padding='valid',activation='tanh')(x_gru)
    x_gru_c1d_gp = GlobalMaxPooling1D()(x_gru_c1d)

    inp2 = Input(shape = (emoji_shape,), name = 'emoji')
    x2 = Embedding(voacb_shape_embdd,300,weights = [emoji_embedd], trainable = False)(inp2)
    X2_conv1 = Conv1D(64, 5, kernel_initializer='he_normal', padding='valid', activation='relu')(x2)
    X2_lstm = LSTM(32, kernel_initializer='he_normal', activation='tanh',
             dropout=0.5, return_sequences=True)(X2_conv1)
    x2_flatten = Flatten()(X2_lstm)
    x2_dense =(Dense(16, activation='tanh') (x2_flatten))  
    

    
    x_f = concatenate([x_lstm_c1d_gp, x_gru_c1d_gp])
    x_f = BatchNormalization()(x_f)
    x_f =(Dense(128, activation='tanh') (x_f))    
    x_f = BatchNormalization()(x_f)
    x_f = concatenate([x_f, x2_dense])    
    x_f = (Dense(64, activation='tanh') (x_f))
    x_f = Dense(3, activation = "sigmoid")(x_f)
    model = Model(inputs = [inp, inp2], outputs = x_f)
    return model


## Load Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir("/content/drive/")

In [ ]:
train_data=pd.read_csv('')

In [ ]:
test_data=pd.read_csv('')


## Clean Data

In [ ]:
train_data['cleaned_text'] = train_data.tweet.apply(clean_text)
test_data['cleaned_text'] = test_data.tweet.apply(clean_text)


In [ ]:
# Extract just the emojis present in each tweet in each set
x_train_emoji = extract_emojis(train_data['tweet'])


In [ ]:
x_test_emoji=extract_emojis(test_data['tweet'])

In [ ]:
train_data = transform(train_data)
test_data = transform(test_data)

In [ ]:
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(train_data['sentiment'])

# binary encode
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
train_labels_task2 = onehot_encoder.fit_transform(integer_encoded)

In [ ]:
integer_encoded = label_encoder.fit_transform(test_data['sentiment'])
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
test_labels_task2 = onehot_encoder.fit_transform(integer_encoded)

### Load Embeddings

In [ ]:
MAX_SEQUENCE_LENGTH = 300 #176 # based on our inputs; TODO: remove outliers? dynamically calculate!
MAX_NB_WORDS = len(word2vec_map.vocab)
EMBEDDING_DIM = 300 # w2v, fastText; GloVe=50

In [ ]:
word2vec_map = load_w2v("/content/drive/MyDrive/Sentiment_Analysis_Arabic/Dataset/arabic-news.bin", binary=True)

In [ ]:
emoji2vec_map = load_w2v("/content/drive/MyDrive/Sentiment_Analysis_Arabic/Dataset/emoji2vec.bin",True)


#### Prepare data by transforming vector2index

In [ ]:
# Get the max length of the train tweets
max_tweet_length = train_data.cleaned_text.map(len).max()

# Convert all tweets into sequences of word indices
tokenizer, train_indices,test_indices = encode_text_as_word_indexes(train_data['cleaned_text'],test_data['cleaned_text'],  lower=True)
vocab_size = len(tokenizer.word_counts) + 1
word_to_index = tokenizer.word_index
print("There are %s unique tokens." % len(word_to_index))

# Pad sequences with 0s (can do it post or pre - post works better here)
x_train = pad_sequences(train_indices, maxlen=max_tweet_length, padding="post", truncating="post", value=0.)
x_test = pad_sequences(test_indices, maxlen=max_tweet_length, padding="post", truncating="post", value=0.)



# Get the word to index and the index to word mappings
word_index = tokenizer.word_index
index_to_word = {index: word for word, index in word_index.items()}

# Set the sequence length
SEQUENCE_LENGTH = max_tweet_length


In [ ]:
# Convert all emoji into sequences of word indices
tokenizer_emji, train_indices_emji,test_indices_emji = encode_text_as_word_indexes(train_data['demoji_text'],test_data['demoji_text'],  lower=True)
word_to_index_emji= tokenizer_emji.word_index
print("There are %s unique tokens." % len(word_to_index_emji))

# Pad sequences with 0s (can do it post or pre - post works better here)
x_train_emji = pad_sequences(train_indices_emji, maxlen=train_data.demoji_text.map(len).max(), padding="post", truncating="post", value=0.)
x_test_emji = pad_sequences(test_indices_emji, maxlen=276, padding="post", truncating="post", value=0.)



# Get the word to index and the index to word mappings
word_index_emji = tokenizer_emji.word_index
index_to_word_emji = {index: word for word, index in word_index_emji.items()}



# Model Preparation

In [ ]:
w2v_embedd=get_embedding_matrix(word2vec_map,word_index,300)

In [ ]:
emoji_embedd=get_embedding_matrix(emoji2vec_map,word_index,300)

In [ ]:
model=keras_dl(w2v_embedd,emoji_embedd,w2v_embedd.shape[0],max_tweet_length,train_data.demoji_text.map(len).max())

In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau


In [ ]:
my_optimizer = Adam(lr=0.0001, beta_1=0.9, beta_2=0.99, decay=0.01)
reduceLR = ReduceLROnPlateau(monitor='accuracy', factor=0.1, patience=3, verbose=1)


In [ ]:
model.compile(loss='categorical_crossentropy', optimizer=my_optimizer, metrics=['accuracy',f1_m,precision_m, recall_m])

### Train Model

In [ ]:
# Fit the model on the training data
history = model.fit([x_train,x_train_emji], train_labels_task2, batch_size=32, epochs=500, shuffle=True,
                             validation_split=0.1, verbose=1)

### Test Model

In [ ]:
prediction=model.predict([x_test,x_test_emji])

In [ ]:
argsorted_y = np.argsort(prediction)[:,-1:]

In [ ]:
cm = confusion_matrix(test_labels_task2.argmax(axis=1), argsorted_y)
recall = np.diag(cm) / np.sum(cm, axis = 1)
precision = np.diag(cm) / np.sum(cm, axis = 0)

In [ ]:
np.mean(recall)


In [ ]:
np.mean(precision)


In [ ]:
print(classification_report(test_labels_task2.argmax(axis=1), argsorted_y))